The purpose of this notebook is to see the placement of scooters during Nashville rush hours (7am - 9am and 4pm - 6pm) and their relative location to bus stops in the Promise Zone. With this notebook, I aim to answer the following questions: 
    1. Where is there an accumulation of scooters during these time periods and where may scooter be lacking? 
    2. Where are the optimal areas to keep/move scooters in order to aid with the "last mile"?
    3. What is the optimal number of scooters to have in these areas? 

First, we will import the needed packages. 

In [2]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMapWithTime
import datetime as dt

Import and inspect the promise zone json file and the May, June and July datasets. These datasets are subsets of the stationary scooter files that include data from only the above-mentioned timeframes. I'll start with May and then repeat the steps for the other datasets. 

In [3]:
promise_zone = gpd.read_file('../data/MDHA_Promise_Zones/promise_zone.json')
print(promise_zone.crs)
promise_zone.head( )

epsg:4326


,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe,geometry
0,1,1,1.053728e+08,49364.601403,"POLYGON ((-86.76297 36.19364, -86.76297 36.193..."
1,2,2,7.670585e+07,45226.018917,"POLYGON ((-86.77838 36.14889, -86.77823 36.148..."
2,3,3,2.985488e+08,75207.067487,"POLYGON ((-86.70771 36.16723, -86.70758 36.166..."
3,4,4,2.713638e+08,74348.596054,"POLYGON ((-86.74735 36.13036, -86.74728 36.130..."
4,5,5,3.928178e+08,106661.712102,"POLYGON ((-86.78490 36.23358, -86.78452 36.232..."


In [4]:
may_rush_hour = pd.read_pickle('../data/may_rush_hour.pkl')
may_rush_hour.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,hour,DOW
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,7,2
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,7,2
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,7,2
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,7,2
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,7,2


Create a geometry column for may_rush_hour to be able to create a geodataframe to use with our map.

In [5]:
may_rush_hour['geometry'] = may_rush_hour.apply(lambda x: Point((x.longitude, 
                                                                x.latitude)), 
                                                                    axis=1)
may_rush_hour.head(3)

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,hour,DOW,geometry
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,7,2,POINT (-86.77045 36.12148)
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,7,2,POINT (-86.770228 36.121393)
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,7,2,POINT (-86.81153999999999 36.144292)


Now, I'll create a geodataframe for the May data that matched the CRS on of the Promise Zone dataframe.

In [8]:
may_geo = gpd.GeoDataFrame(may_rush_hour, 
                           crs = promise_zone.crs, 
                           geometry = may_rush_hour['geometry'])
may_geo.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,hour,DOW,geometry
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,7,2,POINT (-86.77045 36.12148)
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,7,2,POINT (-86.77023 36.12139)
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,7,2,POINT (-86.81154 36.14429)
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,7,2,POINT (-86.77033 36.12162)
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,7,2,POINT (-86.77009 36.12157)


I want to subset the may_geo dataframe into 2 new dataframes containing the morning scooters and the evening scooters. The reasoning behind this is because theoretically we would want scooters available near neighborhoods in the morning to facilitate rides to bus stops, and near bus stops in the evenings to facilitate rides home. 

In [16]:
may_geo_morning = may_geo[may_geo.hour <= 9]
may_geo_morning.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,hour,DOW,geometry
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,7,2,POINT (-86.77045 36.12148)
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,7,2,POINT (-86.77023 36.12139)
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,7,2,POINT (-86.81154 36.14429)
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,7,2,POINT (-86.77033 36.12162)
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,7,2,POINT (-86.77009 36.12157)


In [17]:
may_geo_evening = may_geo[may_geo.hour >= 16]
may_geo_evening.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,hour,DOW,geometry
350206,2019-05-01 16:00:23.173,36.148985,-86.760077,Powered715,41.0,2,16,2,POINT (-86.76008 36.14899)
350207,2019-05-01 16:00:23.173,36.143034,-86.791904,Powered500,83.0,2,16,2,POINT (-86.79190 36.14303)
350208,2019-05-01 16:00:23.173,36.121464,-86.770200,Powered801,49.0,2,16,2,POINT (-86.77020 36.12146)
350209,2019-05-01 16:00:23.173,36.121346,-86.770340,Powered653,80.0,2,16,2,POINT (-86.77034 36.12135)
350210,2019-05-01 16:00:23.173,36.127314,-86.789123,Powered660,98.0,2,16,2,POINT (-86.78912 36.12731)


Now that the May data frame is done, I will read in the bus stops data, create a geodataframe for it, and use a spatial join to merge it with the Promise Zone dataframe, as we only want to show bus stops in this area. 

In [9]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv')
bus_stops.head()

,stop,route,location,lat,lng
0,GREEN LN & WHITES CREEK PIKE WB,GOLDEN VALLEY,"(36.236249, -86.816722)",36.236249,-86.816722
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897
2,DONELSON/DELL STATION OUTBOUND,MURFREESBORO PIKE,"(36.105615, -86.672004)",36.105615,-86.672004
3,17TH AVE S & DOROTHY PL SB,BELMONT,"(36.137623, -86.795609)",36.137623,-86.795609
4,COCKRILL ST & 14TH AVE N,ST. CECILIA - CUMBERLAND,"(36.175944, -86.804242)",36.175944,-86.804242


Create the geometry for the bus stops dataframe.

In [10]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((x.lng, 
                                                         x.lat)), 
                                        axis=1)
bus_stops.head(3)

,stop,route,location,lat,lng,geometry
0,GREEN LN & WHITES CREEK PIKE WB,GOLDEN VALLEY,"(36.236249, -86.816722)",36.236249,-86.816722,POINT (-86.816722 36.236249)
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897,POINT (-86.780897 36.142642)
2,DONELSON/DELL STATION OUTBOUND,MURFREESBORO PIKE,"(36.105615, -86.672004)",36.105615,-86.672004,POINT (-86.672004 36.105615)


Create a geodatafram of the bus stops data and match the CRS to the Promise Zone geodataframe.

In [12]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = promise_zone.crs, 
                           geometry = bus_stops['geometry'])
bus_geo.head()

,stop,route,location,lat,lng,geometry
0,GREEN LN & WHITES CREEK PIKE WB,GOLDEN VALLEY,"(36.236249, -86.816722)",36.236249,-86.816722,POINT (-86.81672 36.23625)
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897,POINT (-86.78090 36.14264)
2,DONELSON/DELL STATION OUTBOUND,MURFREESBORO PIKE,"(36.105615, -86.672004)",36.105615,-86.672004,POINT (-86.67200 36.10562)
3,17TH AVE S & DOROTHY PL SB,BELMONT,"(36.137623, -86.795609)",36.137623,-86.795609,POINT (-86.79561 36.13762)
4,COCKRILL ST & 14TH AVE N,ST. CECILIA - CUMBERLAND,"(36.175944, -86.804242)",36.175944,-86.804242,POINT (-86.80424 36.17594)


Perform a spatial join on the bus_geo and the promise_zone dataframes.

In [13]:
stops_in_promise = gpd.sjoin(bus_geo, promise_zone, op = 'within')
stops_in_promise.head()

,stop,route,location,lat,lng,geometry,index_right,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897,POINT (-86.78090 36.14264),1,2,2,7.670585e+07,45226.018917
3,17TH AVE S & DOROTHY PL SB,BELMONT,"(36.137623, -86.795609)",36.137623,-86.795609,POINT (-86.79561 36.13762),1,2,2,7.670585e+07,45226.018917
5,WEDGEWOOD AVE & 18TH AVE S WB,UNIVERSITY CONNECTOR,"(36.137212, -86.796865)",36.137212,-86.796865,POINT (-86.79686 36.13721),1,2,2,7.670585e+07,45226.018917
53,WEDGEWOOD AVE & 17TH AVE S EB,UNIVERSITY CONNECTOR,"(36.136868, -86.795931)",36.136868,-86.795931,POINT (-86.79593 36.13687),1,2,2,7.670585e+07,45226.018917
61,WEDGEWOOD AVE & 16TH AVE WB,RIVERGATE EXPRESS,"(36.136319, -86.794443)",36.136319,-86.794443,POINT (-86.79444 36.13632),1,2,2,7.670585e+07,45226.018917
